<h1 style="color: #ffffff; background-color: #333333; padding: 10px; border-radius: 10px; text-align: center; text-shadow: 3px 3px 9px #ccc; font-weight: bold;">
  StegoDES: Image Encryption and Steganography with DES and LSB
</h1>

![Steganography](https://media.wired.com/photos/594db1736b76bb7625b89e48/master/pass/hidden_data-01.png)

### Imports
The necessary libraries are imported to handle image processing, mathematical operations, and encryption. These include:
- `numpy` for numerical computations and array manipulation.
- `Pillow` (`PIL`) for image processing.
- `cv2` for image manipulation (though not actively used in this code).
- `math` for PSNR calculation.

In [1]:
import numpy as np
from PIL import Image
import cv2
import math

### Image Loading and Preprocessing
This function:
1. Loads a grayscale image and ensures it is in grayscale format.
2. Loads an RGB image and ensures it is in RGB format.
3. Checks the size compatibility to ensure the grayscale image can be embedded into the RGB image.

In [2]:
# Function to load and preprocess images
def load_images(gray_path, rgb_path):
    # Load grayscale image
    gray_image = Image.open(gray_path).convert('L')  # Convert to grayscale
    gray_array = np.array(gray_image)

    # Load RGB image
    rgb_image = Image.open(rgb_path).convert('RGB')  # Ensure RGB mode
    rgb_array = np.array(rgb_image)

    # Check size compatibility
    if gray_array.size * 8 > rgb_array.size // 3:
        raise ValueError("Grayscale image is too large to be hidden in the RGB image.")

    return gray_array, rgb_array

### DES Tables and Utility Functions
This section defines:
1. Tables (`IP`, `FP`, `E`, `S_BOXES`, etc.) used in DES for permutations and substitutions.
2. Utility functions for bit manipulation, including `permute`, `shift_left`, and `s_box_substitution`.

In [3]:
# DES encryption (complete implementation)
# Initial Permutation Table
IP = [
    58, 50, 42, 34, 26, 18, 10, 2,
    60, 52, 44, 36, 28, 20, 12, 4,
    62, 54, 46, 38, 30, 22, 14, 6,
    64, 56, 48, 40, 32, 24, 16, 8,
    57, 49, 41, 33, 25, 17, 9, 1,
    59, 51, 43, 35, 27, 19, 11, 3,
    61, 53, 45, 37, 29, 21, 13, 5,
    63, 55, 47, 39, 31, 23, 15, 7
]

# Final Permutation Table
FP = [
    40, 8, 48, 16, 56, 24, 64, 32,
    39, 7, 47, 15, 55, 23, 63, 31,
    38, 6, 46, 14, 54, 22, 62, 30,
    37, 5, 45, 13, 53, 21, 61, 29,
    36, 4, 44, 12, 52, 20, 60, 28,
    35, 3, 43, 11, 51, 19, 59, 27,
    34, 2, 42, 10, 50, 18, 58, 26,
    33, 1, 41, 9, 49, 17, 57, 25
]

# Expansion Table
E = [
    32, 1, 2, 3, 4, 5,
    4, 5, 6, 7, 8, 9,
    8, 9, 10, 11, 12, 13,
    12, 13, 14, 15, 16, 17,
    16, 17, 18, 19, 20, 21,
    20, 21, 22, 23, 24, 25,
    24, 25, 26, 27, 28, 29,
    28, 29, 30, 31, 32, 1
]

# S-boxes
S_BOXES = [
    [
        [14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
        [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
        [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
        [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]
    ],
    [
        [15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
        [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
        [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
        [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]
    ],
    [
        [10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
        [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
        [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
        [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]
    ],
    [
        [7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
        [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
        [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
        [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14]
    ],
    [
        [2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
        [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
        [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
        [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3]
    ],
    [
        [12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
        [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
        [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
        [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13]
    ],
    [
        [4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
        [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
        [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
        [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12]
    ],
    [
        [13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
        [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
        [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
        [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11]
    ]
]

# Permutation Table
P = [
    16, 7, 20, 21,
    29, 12, 28, 17,
    1, 15, 23, 26,
    5, 18, 31, 10,
    2, 8, 24, 14,
    32, 27, 3, 9,
    19, 13, 30, 6,
    22, 11, 4, 25
]

# Key Compression Table
PC1 = [
    57, 49, 41, 33, 25, 17, 9,
    1, 58, 50, 42, 34, 26, 18,
    10, 2, 59, 51, 43, 35, 27,
    19, 11, 3, 60, 52, 44, 36,
    63, 55, 47, 39, 31, 23, 15,
    7, 62, 54, 46, 38, 30, 22,
    14, 6, 61, 53, 45, 37, 29,
    21, 13, 5, 28, 20, 12, 4
]

PC2 = [
    14, 17, 11, 24, 1, 5,
    3, 28, 15, 6, 21, 10,
    23, 19, 12, 4, 26, 8,
    16, 7, 27, 20, 13, 2,
    41, 52, 31, 37, 47, 55,
    30, 40, 51, 45, 33, 48,
    44, 49, 39, 56, 34, 53,
    46, 42, 50, 36, 29, 32
]

# Shifts for Key Schedule
SHIFT_SCHEDULE = [1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1]

def permute(block, table):
    return [block[x - 1] for x in table]

def shift_left(block, shifts):
    return block[shifts:] + block[:shifts]

def s_box_substitution(bits):
    output = []
    for i in range(8):
        block = bits[i * 6:(i + 1) * 6]
        row = int(f"{block[0]}{block[-1]}", 2)
        column = int("".join(map(str, block[1:5])), 2)
        s_value = S_BOXES[i][row][column]
        output.extend(map(int, f"{s_value:04b}"))
    return output

### DES Key and Block Functions
Functions for:
1. Subkey generation: `generate_subkeys`.
2. DES block encryption: `des_encrypt_block`.
3. DES block decryption: `des_decrypt_block`.

In [4]:
def generate_subkeys(key):
    key = permute(key, PC1)
    left, right = key[:28], key[28:]
    subkeys = []
    for shift in SHIFT_SCHEDULE:
        left, right = shift_left(left, shift), shift_left(right, shift)
        subkeys.append(permute(left + right, PC2))
    return subkeys

def des_encrypt_block(block, subkeys):
    block = permute(block, IP)
    left, right = block[:32], block[32:]
    for subkey in subkeys:
        expanded_right = permute(right, E)
        xor_result = [a ^ b for a, b in zip(expanded_right, subkey)]
        s_box_result = s_box_substitution(xor_result)
        p_result = permute(s_box_result, P)
        left, right = right, [l ^ p for l, p in zip(left, p_result)]
    return permute(right + left, FP)

def des_decrypt_block(block, subkeys):
    return des_encrypt_block(block, subkeys[::-1])

### DES Encryption and Decryption
Handles:
1. Padding the data.
2. Splitting data into 64-bit blocks.
3. Encrypting or decrypting each block using the DES algorithm.

In [5]:
def encrypt_des(data, key):
    key_bits = list(map(int, f"{int.from_bytes(key, 'big'):064b}"))
    subkeys = generate_subkeys(key_bits)

    # Pad data to be a multiple of 8 bytes (64 bits)
    padding_length = (8 - len(data) % 8) % 8
    data += b" " * padding_length

    encrypted_data = b''
    for i in range(0, len(data), 8):
        block = list(map(int, f"{int.from_bytes(data[i:i+8], 'big'):064b}"))
        encrypted_block = des_encrypt_block(block, subkeys)
        encrypted_data += int("".join(map(str, encrypted_block)), 2).to_bytes(8, 'big')

    return encrypted_data, key

def decrypt_des(encrypted_data, key):
    key_bits = list(map(int, f"{int.from_bytes(key, 'big'):064b}"))
    subkeys = generate_subkeys(key_bits)

    decrypted_data = b''
    for i in range(0, len(encrypted_data), 8):
        block = list(map(int, f"{int.from_bytes(encrypted_data[i:i+8], 'big'):064b}"))
        decrypted_block = des_decrypt_block(block, subkeys)
        decrypted_data += int("".join(map(str, decrypted_block)), 2).to_bytes(8, 'big')

    return decrypted_data.rstrip(b" ")

### LSB Embedding and Extraction
- `embed_lsb`: Embeds encrypted grayscale data into the LSBs of the RGB image.
- `extract_lsb`: Extracts the embedded data from the RGB image.

In [6]:
# Embed grayscale image into RGB image using 1-bit LSB
def embed_lsb(rgb_image, encrypted_bits):
    stego_image = rgb_image.copy()
    height, width, _ = stego_image.shape
    bit_index = 0

    for i in range(height):
        for j in range(width):
            for k in range(3):  # Iterate over R, G, B channels
                if bit_index < len(encrypted_bits):
                    # Modify the least significant bit
                    stego_image[i, j, k] = (stego_image[i, j, k] & 0xFE) | encrypted_bits[bit_index]
                    bit_index += 1
                else:
                    break
    return stego_image

# Extract grayscale image bits from RGB image
def extract_lsb(stego_image, num_bits):
    extracted_bits = []
    height, width, _ = stego_image.shape

    for i in range(height):
        for j in range(width):
            for k in range(3):  # Iterate over R, G, B channels
                if len(extracted_bits) < num_bits:
                    extracted_bits.append(stego_image[i, j, k] & 1)  # Get LSB
                else:
                    break
    return np.array(extracted_bits, dtype=np.uint8)

### PSNR Calculation

The **Peak Signal-to-Noise Ratio (PSNR)** is a mathematical formula used to measure the quality of an image, comparing an original image to a reconstructed or altered version (e.g., a stego image). The equation is as follows:

![PSNR Equation](https://img-blog.csdnimg.cn/20210602153952367.png)

### Explanation:
- **PSNR**: Peak Signal-to-Noise Ratio (in decibels, dB).
- **MAX**: The maximum possible pixel value of the image. For 8-bit images, **MAX** = 255.
- **MSE**: Mean Squared Error between the original and stego images.

In [7]:
# PSNR calculation
def calculate_psnr(original, stego):
    mse = np.mean((original - stego) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = 255.0
    psnr = 10 * math.log10((max_pixel ** 2) / mse)
    return psnr

### Main Execution Flow
The main function:
1. Loads the images.
2. Encrypts the grayscale image.
3. Embeds the encrypted data into the RGB image.
4. Extracts and decrypts the data.
5. Calculates PSNR and saves the output images.

In [8]:
# Main Execution Flow
def main():
    # Paths to the images
    gray_path = r'demo_images\gray_image.png'  # Replace with your path
    rgb_path = r'demo_images\rgb_image.jpg'    # Replace with your path

    # Load images
    gray_image, rgb_image = load_images(gray_path, rgb_path)

    # Flatten grayscale image and encrypt using DES
    gray_flattened = gray_image.flatten()
    key = b'sample12'  # 8-byte DES key
    encrypted_gray, des_key = encrypt_des(gray_flattened.tobytes(), key)

    # Convert encrypted grayscale image to bits
    encrypted_bits = np.unpackbits(np.frombuffer(encrypted_gray, dtype=np.uint8))

    # Embed grayscale image into RGB image
    stego_image = embed_lsb(rgb_image, encrypted_bits)

    # Extract grayscale image bits from stego RGB image
    extracted_bits = extract_lsb(stego_image, len(encrypted_bits))

    # Convert bits back to encrypted grayscale image
    extracted_encrypted_gray = np.packbits(extracted_bits).tobytes()
    decrypted_gray = decrypt_des(extracted_encrypted_gray, des_key)

    # Reshape decrypted grayscale image
    decrypted_gray_image = np.frombuffer(decrypted_gray, dtype=np.uint8).reshape(gray_image.shape)

    # Calculate PSNR
    psnr_value = calculate_psnr(rgb_image, stego_image)

    # Output
    print(f"PSNR between original and stego image: {psnr_value} dB")

    # Save images
    Image.fromarray(stego_image).save(r'demo_images\stego_image.png')
    Image.fromarray(decrypted_gray_image).save(r'demo_images\decrypted_gray_image.png')

In [9]:
if __name__ == "__main__":
    main()

PSNR between original and stego image: 63.04227780786284 dB
